## Funksjonalitet i forhold til eksisterende FDM pakke
Dette er en notebook hvor vi prøver å oppnå samme eller lignende funksjonalitet som finnes i HecoFDMUsage-notebook'en som er basert på Kalles FDM-pakke.

Take-aways:
* API-nøkkel: du må lage en API-nøkkel og lagrer den i Azure KeyVault eller som miljøvariabel på PC-den din (OBS: nøkkelen varer i maks 365 dager)
    - API-nøkkel kan generes her: https://ecofdmprod.lilacenergy.com/Identity/Account/Login
* Output er typisk en liste som inneholder flere ordbøker, f.eks. en ordbok per valueDate
* Hvis du angir fcDate, da må du velge en eksisterende fcDate eller bruke "LatestForecast"-querien
* Ingen MultiCurve-henting


API-dokumentasjon finnes her: https://ecofdmprod.lilacenergy.com/api/swagger/index.html

In [1]:
# Import av relevante pakker
import pandas as pd 
import numpy as np
import os
import requests
import json
from datetime import datetime as dt
from pprint import pprint as pp

#### Hvis du har en personlig KeyVault (Azure), kan du bruke neste boks.

In [2]:
# # Pakker som må installeres
# from azure.keyvault.secrets import SecretClient
# from azure.identity import DefaultAzureCredential

# # Get keyvault name and create connection url
# keyVaultName = os.environ["KEY_VAULT_NAME"]
# KVUri = f"https://{keyVaultName}.vault.azure.net"

# ## DefaultAzureCredential uses the environment variables created earlier to create a credential objects against Azure
# credential = DefaultAzureCredential()

# ## Create an KeyVault secrets client object
# client = SecretClient(vault_url=KVUri, credential=credential)

# ## Bruk string'en som du har lagret i din private Azure KeyVault
# api = client.get_secret("FDM-private-API").value

### Hvis du IKKE har en personlig KeyVault (Azure), lagre FDM_API_KEY som miljøvariabel på PCen din
- Søk på `Rediger miljøvariabler for kontoen din`
- Legg til ny variabel under `Brukervariabler for XXXXX`
    - Variabelnavn: `FDM_API_KEY`
    - Variabelverdi: hvis du har en FDM-bruker kan du opprette en egen API-nøkkel, hvis ikke: spør Niklas

Klikk `Ok` 2x ganger slik at vinduet for miljøvariabler er lukket igjen.

Når du kjører neste boks og får følgende feilmelding: `'NoneType' object has no attribute 'startswith'` -> lukk terminalen / konsolen din og start den opp igjen

In [2]:
# Hent FDM API nøkkelen fra miljøvariablene dine
api = os.getenv('FDM_API_KEY')
if not api.startswith('apikey-v1'):
    api = 'apikey-v1 ' + api

## CurveValues
- kun valueDate og values

In [7]:
# Input
dt_from = '2021-01-10'
dt_to = '2021-01-20'
curveID = '102196349' 

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveValues/{curveID}?MinValueDate={dt_from}&MaxValueDate={dt_to}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()

print(f'Output kommer i form av en liste med ordbøker: {data[0].keys()}\n')

# Få data i en pandas DataFrame
values = [x['value'] for x in data]
valueDates = [x['valueDate'] for x in data]
df = pd.DataFrame(data = values, index = valueDates, columns = ['value'])
df.index = pd.to_datetime(df.index)

print(df.head(4),'\n')

# # Få data i en numpy recarray
# dtype = [('timestamps', 'datetime64[s]'), ('values', 'd')]
# # Fra pandas dataframe to numpy recarray
# data_array = np.rec.array((df.index[:], df.value[:]), dtype=dtype,)

# # Directly into numpy recarray
# zipped_list = [(i,j) for i,j in zip(valueDates, values)]
# data_array = np.rec.array((zipped_list), dtype=dtype,)
# print(data_array[:4])

Output kommer i form av en liste med ordbøker: dict_keys(['scenarioID', 'forecastDate', 'valueDate', 'value'])

                           value
2021-01-10 01:00:00+01:00  35.62
2021-01-10 02:00:00+01:00  33.81
2021-01-10 03:00:00+01:00  33.04
2021-01-10 04:00:00+01:00  32.96 



## CurveValues
- inclCurveInfo = True

In [12]:
## Obligatorisk input
curveID = '102196349' 

## Valgfri input
dt_from = '2021-01-10'
dt_to = '2021-01-20'

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveValues/{curveID}?MinValueDate={dt_from}&MaxValueDate={dt_to}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()

# Få data i en pandas DataFrame
values = [x['value'] for x in data]
valueDates = [x['valueDate'] for x in data]
scenarioID = [x['scenarioID'] for x in data]
forecastDate = [x['forecastDate'] for x in data]
df = pd.DataFrame(data = zip(values, scenarioID, forecastDate), index = valueDates, columns = ['value', 'scenaorioID', 'forecastDate'])
df.index = pd.to_datetime(df.index)
print(df.head(4))

                           value  scenaorioID               forecastDate
2021-01-10 01:00:00+01:00  35.62            0  2000-01-01T01:00:00+01:00
2021-01-10 02:00:00+01:00  33.81            0  2000-01-01T01:00:00+01:00
2021-01-10 03:00:00+01:00  33.04            0  2000-01-01T01:00:00+01:00
2021-01-10 04:00:00+01:00  32.96            0  2000-01-01T01:00:00+01:00


## CurveValues
- Endre tidssone 

OBS: Du kan oppnå samme resultat med "GET/v1/Forecast/{CurveID}/{ScenarioID}/{ForecastDate}

In [13]:
## Obligatorisk input
curveID = '102196349'

## Valgfri input
dt_from = '2021-05-01'
dt_to = '2021-05-05'
## API dokumentasjon: Reference Data > GET/v1/Timezones for å se hvordan tzonene heter
tzone = ['UTC', 'Central European Standard Time'] # Central European Standard Time

for i in tzone:
    # Hent data
    url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveValues/{curveID}?MinValueDate={dt_from}&MaxValueDate={dt_to}&ResultTimezone={i}'
    headers = {"Accept": "application/json", "Authorization": api}
    req = requests.get(url, headers=headers)
    data = req.json()

    # Få data i en pandas DataFrame
    values = [x['value'] for x in data]
    valueDates = [x['valueDate'] for x in data]

    df = pd.DataFrame(data = values, index = valueDates, columns = ['value'])
    df.index = pd.to_datetime(df.index)

    print(f'Timezone {i} results in:\n{df.head(5)}\n')

Timezone UTC results in:
                           value
2021-05-01 00:00:00+00:00  48.00
2021-05-01 01:00:00+00:00  48.00
2021-05-01 02:00:00+00:00  48.15
2021-05-01 03:00:00+00:00  48.38
2021-05-01 04:00:00+00:00  48.84

Timezone Central European Standard Time results in:
                           value
2021-05-01 02:00:00+02:00  48.00
2021-05-01 03:00:00+02:00  48.00
2021-05-01 04:00:00+02:00  48.15
2021-05-01 05:00:00+02:00  48.38
2021-05-01 06:00:00+02:00  48.84



## CurveValues
- Endre tidsfrekvens

In [14]:
## Obligatorisk input
curveID = '102196349'
# API dokumentasjone: Reference Data > GET/v1/PeriodTypes for å se hvilke aggregeringer som er mulig
ResultPeriodType = 'Days'
ResultPeriodLength = 5 # over hvor mange "enheter" (f.eks. dager) skal det aggregeres

## Valgfri input
MinValueDate = '2021-01-01'
# ...

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveValues/Aggregated/{curveID}/{ResultPeriodType}/{ResultPeriodLength}?MinValueDate={MinValueDate}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()

# Få data i en pandas DataFrame
values = [x['value'] for x in data]
valueDates = [x['valueDate'] for x in data]
valueEndDate = [x['valueEndDate'] for x in data]

df = pd.DataFrame(data = zip(valueEndDate, values), index = valueDates, columns = ['valueEndDate', 'values'])
df.index = pd.to_datetime(df.index)

print(df.head(5))

                                        valueEndDate     values
2021-01-01 00:00:00+01:00  2021-01-06T00:00:00+01:00  33.164250
2021-01-06 00:00:00+01:00  2021-01-11T00:00:00+01:00  53.488750
2021-01-11 00:00:00+01:00  2021-01-16T00:00:00+01:00  48.822833
2021-01-16 00:00:00+01:00  2021-01-21T00:00:00+01:00  46.963750
2021-01-21 00:00:00+01:00  2021-01-26T00:00:00+01:00  38.817417


## CurveValues
* Div med fcDate
    - AsAtDate: returnerer den nyeste fcDate før AsAtDate. Det betyr at det må finnes minst en kurve med fcDate før AsAtDate.


OBS: du må enten velge en eksisterende fcDate eller bruk "LatestForecast"

In [15]:
## Obligatorisk input
curveID = '910000065'
scenarioID = 0

## Valgfri input
AsAtDate = '2020-01-01'
# ...

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveValues/LatestForecast/{curveID}/{scenarioID}?AsAtDate={AsAtDate}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()

# Få data i en pandas DataFrame
values = [x['value'] for x in data]
valueDates = [x['valueDate'] for x in data]

df = pd.DataFrame(data = values, index = valueDates, columns = ['value'])
df.index = pd.to_datetime(df.index)

print(f'With AsAtDate set at {AsAtDate}:\n{df.head(5)}\n')

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveValues/LatestForecast/{curveID}/{scenarioID}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()

# Få data i en pandas DataFrame
values = [x['value'] for x in data]
valueDates = [x['valueDate'] for x in data]

df = pd.DataFrame(data = values, index = valueDates, columns = ['value'])
df.index = pd.to_datetime(df.index)

print(f'With no AsAtDate:\n{df.head(5)}')

With AsAtDate set at 2020-01-01:
                              value
2019-12-31 00:00:00+01:00  32.86442
2019-12-31 01:00:00+01:00  32.51643
2019-12-31 02:00:00+01:00  32.36680
2019-12-31 03:00:00+01:00  32.33875
2019-12-31 04:00:00+01:00  32.33875

With no AsAtDate:
                               value
2022-06-10 00:00:00+02:00  136.91201
2022-06-10 01:00:00+02:00  134.43276
2022-06-10 02:00:00+02:00  134.46350
2022-06-10 03:00:00+02:00  135.01672
2022-06-10 04:00:00+02:00  135.04745


## Curve Summary
- Finn ut hvilke fcDates det finnes per kurve

In [16]:
## Obligatorisk input
curveID = '910000065'

## Valgfri input
scenarioID = 0
MinForecastDate = '2021-01-01'
# ...

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveForecastList/{curveID}?ScenarioID={scenarioID}&MinForecastDate={MinForecastDate}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)

data = req.json()
pp(data[:5])

[{'curveID': 910000065,
  'forecastDate': '2021-01-04T09:28:07+00:00',
  'lastUpdateTime': '2021-01-04T08:28:17.757+00:00',
  'maxValueDate': '2023-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0},
 {'curveID': 910000065,
  'forecastDate': '2021-01-04T14:15:47+00:00',
  'lastUpdateTime': '2021-01-04T13:17:13.233+00:00',
  'maxValueDate': '2023-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0},
 {'curveID': 910000065,
  'forecastDate': '2021-01-04T14:32:51+00:00',
  'lastUpdateTime': '2021-01-04T13:33:08.41+00:00',
  'maxValueDate': '2023-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0},
 {'curveID': 910000065,
  'forecastDate': '2021-01-04T14:37:58+00:00',
  'lastUpdateTime': '2021-01-04T13:38:43.727+00:00',
  'maxValueDate': '2024-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0},
 {'curveID': 910000065,
  'forecastDate': '2021-0

In [17]:
## Obligatorisk input
curveID = '910000065'

## Valgfri input
scenarioID = 0
MinForecastDate = '2021-01-01'
# ...

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveForecastList/{curveID}?ScenarioID={scenarioID}&MinForecastDate={MinForecastDate}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)

data = req.json()
pp(data[:3])

[{'curveID': 910000065,
  'forecastDate': '2021-01-04T09:28:07+00:00',
  'lastUpdateTime': '2021-01-04T08:28:17.757+00:00',
  'maxValueDate': '2023-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0},
 {'curveID': 910000065,
  'forecastDate': '2021-01-04T14:15:47+00:00',
  'lastUpdateTime': '2021-01-04T13:17:13.233+00:00',
  'maxValueDate': '2023-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0},
 {'curveID': 910000065,
  'forecastDate': '2021-01-04T14:32:51+00:00',
  'lastUpdateTime': '2021-01-04T13:33:08.41+00:00',
  'maxValueDate': '2023-12-31T22:00:00+00:00',
  'minValueDate': '2021-01-04T23:00:00+00:00',
  'scenarioID': 0}]


## Curve Summary
- Finn ut hvor mange fcDates det finnes for en en serie

Det finnes lignende queries for å finne ut om minValueDate, maxvalueDate, lastUpdateTime og scenarier.

In [18]:
## Obligatorisk input
curveID = '910000065'

# Hent data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/CurveSummary/Forecasts/{curveID}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()
print(data)

{'curveID': 910000065, 'forecastCount': 3990, 'minForecastDate': '2015-01-05T00:00:00+00:00', 'maxForecastDate': '2022-06-09T09:01:34+00:00', 'lastUpdateTime': '2022-06-09T07:03:57.923+00:00'}


## Metadata

In [19]:
# Input
curveID = '102196359'

# Hente data
url = f'https://ecofdmprod.lilacenergy.com/api/v1/Metadata/{curveID}'
headers = {"Accept": "application/json", "Authorization": api}
req = requests.get(url, headers=headers)
data = req.json()

print(f'Output kommer i form av en ordbok som inneholder 2x nøkler: {data.keys()}\n')
print('curveID =',data['curveID'], '\n')

# Nøkkelen 'tags' inneholder en liste med ordbøker
for i in data['tags']:
    print(i)

# Hvis du skal ha en spesifikk verdi
for i in range(len(data['tags'])):
    if data['tags'][i]['name'] == 'Source':
        print('\n',f'Datakilde for curveID {curveID} er',data['tags'][i]['value'])

Output kommer i form av en ordbok som inneholder 2x nøkler: dict_keys(['curveID', 'tags'])

curveID = 102196359 

{'name': 'Curve Name', 'value': 'Nordpool-PEUR;Nordpool-PR-D-Bergen'}
{'name': 'Commodity', 'value': 'Power'}
{'name': 'Provider.is', 'value': 'Nord Pool Spot AS'}
{'name': 'Variable.is', 'value': 'Price'}
{'name': 'Geography.is', 'value': 'Bergen (NOR)'}
{'name': 'Value.Frequency.is', 'value': 'Hours (1)'}
{'name': 'Status.is', 'value': 'Actual'}
{'name': 'Unit.is', 'value': 'Euro per megawatt hour'}
{'name': 'Timezone.is', 'value': 'CET'}
{'name': 'FileFolder', 'value': 'History/Prices'}
{'name': 'FileStub', 'value': 'HIST_Pwr_NPS_PRI_Elspot_City_NRD_A'}
{'name': 'FileID', 'value': '5020730'}
{'name': 'Source', 'value': 'PointConnect'}
{'name': 'Commodity.is', 'value': 'Power'}
{'name': 'Provider', 'value': 'Nord Pool Spot AS'}
{'name': 'Unit', 'value': 'EUR/MWh'}
{'name': 'Issue.Frequency.is', 'value': 'Days (1)'}
{'name': 'Origin.is', 'value': 'Nord Pool Spot AS'}
{'nam

## Proprietary Data: write to FDM
* Se repo'et https://dev.azure.com/hafslunde-co/HafslundEco.Krafthandel.FUNdament/_git/heco.kraft.fun.fdmapi dersom du har lyst på en robust løsning for å skrive data til FDM (og annen funksjonalitet)

Test tidsserie:
- curveID: 910001922
- scenarioID: 700